In [1]:
import gzip
import json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

with gzip.open('xbar/1/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))
    
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

def buildBST(array,start=0,finish=-1):
    if finish<0:
        finish = len(array)
    mid = (start + finish) // 2
    if mid-start==1:
        ltl=start
    else:
        ltl=buildBST(array,start,mid)
    
    if finish-mid==1:
        gtl=mid
    else:
        gtl=buildBST(array,mid,finish)
        
    return((array[mid],ltl,gtl))

congestion_data = np.load('xbar/1/xbar_congestion.npz')
xbst=buildBST(congestion_data['xBoundaryList'])
ybst=buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])


def getGRCIndex(x,y,xbst,ybst):
    while (type(xbst)==tuple):
        if x < xbst[0]:
            xbst=xbst[1]
        else:
            xbst=xbst[2]
            
    while (type(ybst)==tuple):
        if y < ybst[0]:
            ybst=ybst[1]
        else:
            ybst=ybst[2]
            
    return ybst, xbst


for k in range(instances.shape[0]):
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
    demand[k] = d
        
instances['routing_demand'] = demand

1a

In [2]:
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix
from sklearn.metrics import mean_squared_error, r2_score
import scipy.sparse as sp


seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

conn=np.load('xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)
A = torch.FloatTensor(np.array(A.todense()))
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx
A = normalize(A)
A = torch.FloatTensor(np.array(A))


target_values = instances['routing_demand'] 

train_size = 0.7
indices = np.arange(len(target_values))
train_indices, test_indices = train_test_split(indices, train_size=train_size, random_state=seed)

scaler = StandardScaler()
features = scaler.fit_transform(instances[['xloc', 'yloc', 'cell', 'orient']]) 


features = torch.FloatTensor(features)
targets = torch.FloatTensor(target_values)

In [3]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.gcn_layers = nn.ModuleList([nn.Linear(input_dim, hidden_dim)])
        for _ in range(self.num_layers - 2):
            self.gcn_layers.append(nn.Linear(hidden_dim, hidden_dim))
        self.gcn_layers.append(nn.Linear(hidden_dim, output_dim))

    def forward(self, x, adj):
        for layer in self.gcn_layers[:-1]:
            x = F.relu(layer(torch.matmul(adj, x)))
        x = self.gcn_layers[-1](torch.matmul(adj, x))
        return x

input_dim = features.shape[1]
output_dim = 1  
hidden_dim = 64
num_epochs = 1000
learning_rate = 0.01



In [4]:
features.shape

torch.Size([3952, 4])

In [5]:
A.shape

torch.Size([3952, 3952])

In [6]:
for k in range(1, 6):
    model = GCN(input_dim, hidden_dim, output_dim, num_layers=k)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(3000):
        model.train()
        optimizer.zero_grad()
        outputs = model(features, A)
        loss = criterion(outputs[train_indices], targets[train_indices])
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 500 == 0:
            print(f"Layer {k}, Epoch {epoch + 1}, Loss: {loss.item()}")

    model.eval()
    with torch.no_grad():
        test_outputs = model(features, A)
        test_loss = criterion(outputs[test_indices], targets[test_indices])
        print(f"Layer {k}, Test Loss: {test_loss.item()}")

/Users/viv/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Layer 1, Epoch 500, Loss: 20.66912078857422
Layer 1, Epoch 1000, Loss: 20.424848556518555
Layer 1, Epoch 1500, Loss: 20.412565231323242
Layer 1, Epoch 2000, Loss: 20.407617568969727
Layer 1, Epoch 2500, Loss: 20.405126571655273
Layer 1, Epoch 3000, Loss: 20.403888702392578
Layer 1, Test Loss: 20.544363021850586


/Users/viv/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1186])) that is different to the input size (torch.Size([1186, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Layer 2, Epoch 500, Loss: 20.73029899597168
Layer 2, Epoch 1000, Loss: 20.47831916809082
Layer 2, Epoch 1500, Loss: 20.425373077392578
Layer 2, Epoch 2000, Loss: 20.410696029663086
Layer 2, Epoch 2500, Loss: 20.405027389526367
Layer 2, Epoch 3000, Loss: 20.402406692504883
Layer 2, Test Loss: 20.541851043701172
Layer 3, Epoch 500, Loss: 20.427040100097656
Layer 3, Epoch 1000, Loss: 20.40473747253418
Layer 3, Epoch 1500, Loss: 20.40302276611328
Layer 3, Epoch 2000, Loss: 20.402299880981445
Layer 3, Epoch 2500, Loss: 20.401920318603516
Layer 3, Epoch 3000, Loss: 20.40167808532715
Layer 3, Test Loss: 20.540998458862305
Layer 4, Epoch 500, Loss: 20.41515350341797
Layer 4, Epoch 1000, Loss: 20.405214309692383
Layer 4, Epoch 1500, Loss: 20.403547286987305
Layer 4, Epoch 2000, Loss: 20.402734756469727
Layer 4, Epoch 2500, Loss: 20.40222930908203
Layer 4, Epoch 3000, Loss: 20.403549194335938
Layer 4, Test Loss: 20.543567657470703
Layer 5, Epoch 500, Loss: 20.41018295288086
Layer 5, Epoch 1000, 

1b

In [7]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.gcn_layers = nn.ModuleList([nn.Linear(input_dim, hidden_dim)])
        self.attention_weights = nn.ModuleList([nn.Linear(hidden_dim * 2, 1)])
        for _ in range(self.num_layers - 2):
            self.gcn_layers.append(nn.Linear(hidden_dim, hidden_dim))
            self.attention_weights.append(nn.Linear(hidden_dim * 2, 1))
        self.gcn_layers.append(nn.Linear(hidden_dim, output_dim))

    def forward(self, x, adj):
        for layer in self.gcn_layers[:-1]:
            x = F.relu(layer(torch.matmul(adj, x)))
        for layer_idx in range(self.num_layers - 1):
            attention_scores = self.attention_weights[layer_idx](torch.cat((x, torch.matmul(adj, x)), dim=1))
            attention_scores = F.softmax(attention_scores, dim=0)
            x = x * attention_scores + torch.matmul(adj, x) * (1 - attention_scores)
        x = self.gcn_layers[-1](torch.matmul(adj, x))
        return x


input_dim = features.shape[1]
output_dim = 1  
hidden_dim = 64
num_epochs = 1000
learning_rate = 0.01



In [8]:
for k in range(1, 6):
    model = GCN(input_dim, hidden_dim, output_dim, num_layers=k)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(3000):
        model.train()
        optimizer.zero_grad()
        outputs = model(features, A)
        loss = criterion(outputs[train_indices], targets[train_indices])
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 500 == 0:
            print(f"Layer {k}, Epoch {epoch + 1}, Loss: {loss.item()}")


    model.eval()
    with torch.no_grad():
        test_outputs = model(features, A)
        test_loss = criterion(outputs[test_indices], targets[test_indices])
        print(f"Layer {k}, Test Loss: {test_loss.item()}")

Layer 1, Epoch 500, Loss: 20.625022888183594
Layer 1, Epoch 1000, Loss: 20.45734214782715
Layer 1, Epoch 1500, Loss: 20.418825149536133
Layer 1, Epoch 2000, Loss: 20.40803337097168
Layer 1, Epoch 2500, Loss: 20.404417037963867
Layer 1, Epoch 3000, Loss: 20.402830123901367
Layer 1, Test Loss: 20.54224967956543
Layer 2, Epoch 500, Loss: 20.715463638305664
Layer 2, Epoch 1000, Loss: 20.438764572143555
Layer 2, Epoch 1500, Loss: 20.413219451904297
Layer 2, Epoch 2000, Loss: 20.406387329101562
Layer 2, Epoch 2500, Loss: 20.40394401550293
Layer 2, Epoch 3000, Loss: 20.40288543701172
Layer 2, Test Loss: 20.542524337768555
Layer 3, Epoch 500, Loss: 20.404150009155273
Layer 3, Epoch 1000, Loss: 20.40196418762207
Layer 3, Epoch 1500, Loss: 20.401735305786133
Layer 3, Epoch 2000, Loss: 20.401643753051758
Layer 3, Epoch 2500, Loss: 20.401660919189453
Layer 3, Epoch 3000, Loss: 20.40170669555664
Layer 3, Test Loss: 20.540868759155273
Layer 4, Epoch 500, Loss: 20.41494369506836
Layer 4, Epoch 1000, 

In [11]:
with gzip.open('xbar/2/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))
    
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('xbar/2/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

def buildBST(array,start=0,finish=-1):
    if finish<0:
        finish = len(array)
    mid = (start + finish) // 2
    if mid-start==1:
        ltl=start
    else:
        ltl=buildBST(array,start,mid)
    
    if finish-mid==1:
        gtl=mid
    else:
        gtl=buildBST(array,mid,finish)
        
    return((array[mid],ltl,gtl))

congestion_data = np.load('xbar/2/xbar_congestion.npz')
xbst=buildBST(congestion_data['xBoundaryList'])
ybst=buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])


def getGRCIndex(x,y,xbst,ybst):
    while (type(xbst)==tuple):
        if x < xbst[0]:
            xbst=xbst[1]
        else:
            xbst=xbst[2]
            
    while (type(ybst)==tuple):
        if y < ybst[0]:
            ybst=ybst[1]
        else:
            ybst=ybst[2]
            
    return ybst, xbst


for k in range(instances.shape[0]):
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
    demand[k] = d
        
instances['routing_demand'] = demand

In [12]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

conn=np.load('xbar/2/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)
A = torch.FloatTensor(np.array(A.todense()))
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx
A = normalize(A)
A = torch.FloatTensor(np.array(A))


target_values = instances['routing_demand'] 



In [13]:
train_size = 0.7
indices = np.arange(len(target_values))


scaler = StandardScaler()
features = scaler.fit_transform(instances[['xloc', 'yloc', 'cell', 'orient']]) 


features = torch.FloatTensor(features)
targets = torch.FloatTensor(target_values)

In [14]:
input_dim = features.shape[1]
output_dim = 1  
hidden_dim = 64
num_epochs = 1000
learning_rate = 0.01

In [15]:
test_outputs = model(features, A)


In [16]:
test_loss = criterion(test_outputs, targets)


/Users/viv/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([6872])) that is different to the input size (torch.Size([6872, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [20]:
print(f"Test Loss: {test_loss.item()}")

Test Loss: 174.86117553710938
